In [21]:
from sqlalchemy import create_engine
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV
import pickle
import datetime
import numpy as np

In [22]:
user = "erpnext_sql"
password = "rtp[ps4XRF0_pX"
db = "_abde597b15b69fc8"
host = "erpnext.responsible-ai.net"
port = "3333"

In [23]:
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:{port}/{db}")

### Dateset

In [24]:
# Load data from SQL
data_sql_full = pd.read_sql("""
    SELECT 
        po.name, 
        po.supplier, 
        po.total_qty,
        po.set_warehouse,
        po.transaction_date as order_date,
        po_items.schedule_date,
        po_items.item_name,
        po_receipt.posting_date as delivered_date
    FROM `tabPurchase Order` po
    LEFT JOIN `tabPurchase Order Item` po_items
        ON po.name = po_items.parent
    LEFT JOIN `tabPurchase Receipt Item` po_receipt_item
        ON po.name = po_receipt_item.purchase_order 
    LEFT JOIN `tabPurchase Receipt` po_receipt
        ON po_receipt_item.parent = po_receipt.name
    """, engine)


In [25]:
data_sql_full.head()

,name,supplier,total_qty,set_warehouse,order_date,schedule_date,item_name,delivered_date
0,e,Beans Inc.,750.0,Naples - RR,2020-06-14,None,None,None
1,PUR-ORD-2024-08-00002,Beans Inc.,750.0,Naples - RR,2023-04-24,2023-04-29,Excelsa,2023-05-04
2,PUR-ORD-2024-08-00003,Fair Trade AG,750.0,Nairobi - RR,2024-09-08,2024-09-21,Arabica,None
3,PUR-ORD-2024-08-00004,Aromatico,1000.0,Amsterdam - RR,2022-10-26,2022-11-10,Maragogype,2022-11-29
4,PUR-ORD-2024-08-00005,Aromatico,500.0,London - RR,2022-03-18,2022-04-05,Maragogype Type B,2022-04-24


In [26]:
# Drop rows with missing delivered_date or schedule_date
data_sql_full.dropna(inplace=True, subset=["delivered_date", "schedule_date"])

# Convert dates to datetime objects
data_sql_full["delivered_date"] = pd.to_datetime(data_sql_full["delivered_date"])
data_sql_full["schedule_date"] = pd.to_datetime(data_sql_full["schedule_date"])

# Create new features
data_sql_full["late"] = data_sql_full["delivered_date"] > data_sql_full["schedule_date"]
data_sql_full["days_late"] = (data_sql_full["delivered_date"] - data_sql_full["schedule_date"]).dt.days
data_sql_full["schedule_month"] = data_sql_full["schedule_date"].dt.month


In [27]:
# One-hot encode categorical features
d_suppliers = pd.get_dummies(data_sql_full['supplier'], prefix="d_sup")
d_warehouses = pd.get_dummies(data_sql_full['set_warehouse'], prefix="d_wh")
d_item_names = pd.get_dummies(data_sql_full['item_name'], prefix="d_item")

# Merge dummy variables into the main dataset
data_for_model = pd.concat([data_sql_full, d_suppliers, d_warehouses, d_item_names], axis=1)


## Train / Test split

In [28]:
# Define feature list for the model
features = ["total_qty", "schedule_month"] + list(d_suppliers.columns) + list(d_warehouses.columns) + list(d_item_names.columns)

# Define the train-test split date
train_split_date = datetime.date(2023, 3, 1)

# Split data into training and test sets
train = data_for_model[data_for_model["order_date"] <= train_split_date]
test = data_for_model[data_for_model["order_date"] > train_split_date]


## 1. XGBoost Classifier Model

In [29]:
# Define parameter grid for GridSearchCV
param_grid = {
    'n_estimators': [100, 300, 500],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7]
}

# Initialize GradientBoostingRegressor
gbm = GradientBoostingRegressor(random_state=1234)

# Grid search with cross-validation
grid_search = GridSearchCV(estimator=gbm, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', verbose=2, n_jobs=-1)
grid_search.fit(train[features], train["days_late"])


Fitting 3 folds for each of 27 candidates, totalling 81 fits


GridSearchCV(cv=3, estimator=GradientBoostingRegressor(random_state=1234),
             n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.05, 0.1],
                         'max_depth': [3, 5, 7],
                         'n_estimators': [100, 300, 500]},
             scoring='neg_mean_squared_error', verbose=2)

In [30]:
# Get the best model from grid search
best_gbm = grid_search.best_estimator_

# Predict on the test data
predictions_gbm = best_gbm.predict(test[features])


In [31]:
# Evaluate the model performance
mse_gbm = mean_squared_error(test["days_late"], predictions_gbm)
rmse_gbm = np.sqrt(mse_gbm)
mae_gbm = mean_absolute_error(test["days_late"], predictions_gbm)

# Print evaluation metrics
print(f"RMSE: {rmse_gbm}")
print(f"MAE: {mae_gbm}")

# Print the best hyperparameters
print(f"Best parameters: {grid_search.best_params_}")


RMSE: 1.694838817096295
MAE: 1.3648708877168478
Best parameters: {'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 300}


### Export Model

In [32]:
# Export the best model as a pickle file
export_name = "best_gbm_model_with_features.pkl"
with open(export_name, "wb") as file:
    pickle.dump(best_gbm, file)

## Random Forest Regressor

In [33]:
from sklearn.ensemble import RandomForestRegressor

# Initialize and train a Random Forest model
rf = RandomForestRegressor(n_estimators=200, random_state=1234)
rf.fit(train[features], train["days_late"])

# Predict and evaluate
predictions_rf = rf.predict(test[features])
mse_rf = mean_squared_error(test["days_late"], predictions_rf)
rmse_rf = np.sqrt(mse_rf)
mae_rf = mean_absolute_error(test["days_late"], predictions_rf)

print(f"Random Forest RMSE: {rmse_rf}")
print(f"Random Forest MAE: {mae_rf}")


Random Forest RMSE: 1.9438252756572554
Random Forest MAE: 1.5392118013568505


## Decision Tree Regressor

In [34]:
from sklearn.tree import DecisionTreeRegressor

# Initialize and train a Decision Tree model
dt = DecisionTreeRegressor(max_depth=5, random_state=1234)
dt.fit(train[features], train["days_late"])

# Predict and evaluate
predictions_dt = dt.predict(test[features])
mse_dt = mean_squared_error(test["days_late"], predictions_dt)
rmse_dt = np.sqrt(mse_dt)
mae_dt = mean_absolute_error(test["days_late"], predictions_dt)

print(f"Decision Tree RMSE: {rmse_dt}")
print(f"Decision Tree MAE: {mae_dt}")


Decision Tree RMSE: 3.1174053670483084
Decision Tree MAE: 2.492790227741088
